# Python Package Import

In [ ]:
import os
import json
import fnmatch
import glob

# BIDS Data Organisation

I first used the scripts code/prebio-organise_tpX.py, which used dcm2niix for dicom import. After testing MRtrix and realising that dcm2nii was introducing a rotation to the DWI vectors (see http://community.mrtrix.org/t/dw-scheme-dicom-import-or-fslgrad-json-import/1961) I re-ran this import for the DWI data using the updated dcm2niix version installed from conda.

In [ ]:
!dcm2niix -v

### DWI Data Import

In [ ]:
# define function that lists all files except hidden files
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

toplvl = "/Volumes/data/prebiostress/data"
niidir = toplvl + "/sourcedata/nifti"
tp="1" # do timepoint1
dcmdir = "/Volumes/data/prebiostress/data/sourcedata/dicom/timepoint"+tp

#RL
for fname in listdir_nohidden(dcmdir):
    subj = fname.split("_")[-2].split("_")[0]

    datadir = glob.glob(dcmdir+'/'+fname+'/*/DWI_MB3_DIR64_RL_0*')
    datadir = ''.join(datadir)
    cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_acq-rl_dwi " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/dwi " +datadir
    print(cmd)
    os.system(cmd)
#LR
for fname in listdir_nohidden(dcmdir):
    subj = fname.split("_")[-2].split("_")[0]

    datadir = glob.glob(dcmdir+'/'+fname+'/*/DWI_MB3_DIR64_LR_0*')
    datadir = ''.join(datadir)
    cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_acq-lr_dwi " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/dwi " +datadir
    print(cmd)
    os.system(cmd)

for tp in ["2", "3", "4"]: # do timepoints 2:4
    dcmdir = "/Volumes/data/prebiostress/data/sourcedata/dicom/timepoint"+tp

    #RL
    for fname in listdir_nohidden(dcmdir):
        subj = fname.split("_")[-3].split("_")[0]

        datadir = glob.glob(dcmdir+'/'+fname+'/*/DWI_MB3_DIR64_RL_0*')
        datadir = ''.join(datadir)
        cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_acq-rl_dwi " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/dwi " +datadir
        print(cmd)
        os.system(cmd)
    #LR
    for fname in listdir_nohidden(dcmdir):
        subj = fname.split("_")[-3].split("_")[0]

        datadir = glob.glob(dcmdir+'/'+fname+'/*/DWI_MB3_DIR64_LR_0*')
        datadir = ''.join(datadir)
        cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_acq-lr_dwi " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/dwi " +datadir
        print(cmd)
        os.system(cmd)

# Testing Whether dcm2niix Gradient Vectors Are Correct

Compare the vecs from the new version of dcm2niix with the older version and that of MRtrix:


In [ ]:
!mrconvert -fslgrad /Volumes/data/prebiostress/data/sourcedata/nifti/test2/sub-24_ses-1_acq-rl_dwi.bvec /Volumes/data/prebiostress/data/sourcedata/nifti/test2/sub-24_ses-1_acq-rl_dwi.bval /Volumes/data/prebiostress/data/sourcedata/nifti/test2/sub-24_ses-1_acq-rl_dwi.nii.gz /Volumes/data/prebiostress/data/sourcedata/nifti/test2/sub-24_ses-1_acq-rl_dwi.mif
!dwi2tensor sub-24_ses-1_acq-rl_dwi.mif - | tensor2metric - -vec dec.mif
!mrview /Volumes/data/prebiostress/data/sourcedata/nifti/test-sub24-ses1-imports/fa_dicom-rl.mif # then open the fixel plot of dec.mif

This worked! The vectors now appear to have the correct orientation and the same as just using MRtrix for dicom import.

# Notes: Brain Mask Decisions
Tried to use a multiatlas skull stripping approach for the prebiostress images. 
However, the Template images I tried to use came from 3 year old sheep and the image content (outside the brain) between these and even the 10 week old lambs was just too different.

The first manual brain mask segmentation was conducted on **sub23_ses4** using both the T1 and T2.
To enable opening both images in the same ITK-SNAP window the T2 was resliced to the T1:


In [ ]:
!c3d /Volumes/data/prebiostress/data/derivatives/sub-23/ses-4/anat/sub-23_ses-4_T1w_dn.nii.gz /Volumes/data/prebiostress/data/derivatives/sub-23/ses-4/anat/sub-23_ses-4_T2w_dn.nii.gz -reslice-identity -o /Volumes/data/prebiostress/data/derivatives/sub-23/ses-4/anat/testManualBrainMask/t2-in-t1space.nii.gz

![alt text](images/Screenshot2018-11-26.png)

Default image contrast values were used.

### Decisions:
* Meninges: Voxels containing meningeal tissue were excluded from the mask. However, when there was a doubt as to whether the voxel's tissue was meningeal or cerebral the benefit of doubt was given to cerebral and the voxel was included in the mask.
* Brainstem: The brainstem segmentation was stopped approximately 5 slices posterior to the last posterior slice containing cerebellum.
* Pituitary Gland: was included in the mask.
* CSF: voxels within the ventricular system and in the sulci were included in the mask.
* Optic tract, nerve & chiasm: were excluded from the mask.
* Olfactory Bulbs: were included in the mask.

# Register 2 10w Images: A First Attempt
Get BIDS folder structure info:

In [ ]:
from bids.grabbids import BIDSLayout

niftipath = '/Volumes/data/prebiostress/data/derivatives'
layout = BIDSLayout(niftipath) # ignore the UserWarning: No valid root....

Get path to the subject 23 session 4 anatomical - it has a manually segmented brain mask.  
Get path to the subject 01 session 4 anatomical - we will try to automatically skull strip it.  
Create brainmask folder for subject 01.

In [ ]:
fixed = layout.get(subject="23",modality="anat",session="4",type="T1w",return_type='file')
fixed = fixed[0]
moving = layout.get(subject="01",modality="anat",session="4",type="T1w",return_type='file')
moving = moving[0]
out_dir = (os.path.dirname(moving)+"/brainmask/")
os.makedirs(out_dir,exist_ok="TRUE")

### Test the initialization stage:

In [ ]:
%%bash -s "$fixed" "$moving" "$out_dir"
antsRegistration -d 3 \
-u 1 \
-t Rigid[0.1] \
-m MI[ $1, $2, 1, 32] \
-c 0 \
-f 4 \
-s 2 \
-o [${3}movingInit_, ${3}movingInit_deformed.nii.gz] 

In [ ]:
%%bash -s "$fixed" "$moving" "$out_dir"
antsRegistration -d 3 \
-u 1 \
-r [ $1, $2, 0] \
-t Rigid[0.1] \
-m MI[ $1, $2, 1, 32] \
-c 0 \
-f 4 \
-s 2 \
-o [${3}movingInit_center, ${3}movingInit_deformed_center.nii.gz] 

In [ ]:
%%bash -s "$fixed" "$moving" "$out_dir"
antsRegistration -d 3 \
-u 1 \
-r [ $1, $2, 1] \
-t Rigid[0.1] \
-m MI[ $1, $2, 1, 32] \
-c 0 \
-f 4 \
-s 2 \
-o [${3}movingInit_intensity, ${3}movingInit_deformed_intensity.nii.gz] 

There was not so much difference between using the center of mass or the intensities at least for this image.
Both improved the registration.

### Test the rigid stage:

In [ ]:
%%bash -s "$fixed" "$moving" "$out_dir"
antsRegistration -d 3 \
-u 1 \
-w [0.005,0.995] \
-r [${3}movingInit_intensity0GenericAffine.mat] \
-t Rigid[0.1] \
-m MI[ $1, $2, 1, 32, Regular,0.2] \
-c [1000x500x250x100,1e-6,10] \
-f 8x4x2x1 \
-s 3x2x1x0vox \
-o [${3}movingRigid, ${3}movingRigid_deformed.nii.gz] 

The rigid stage improved from the initialization. 

### Test the affine stage:

In [ ]:
%%bash -s "$fixed" "$moving" "$out_dir"
antsRegistration -d 3 \
-u 1 \
-w [0.005,0.995] \
-r [${3}movingRigid0GenericAffine.mat] \
-t Affine[0.1] \
-m MI[ $1, $2, 1, 32, Regular,0.2] \
-c [1000x500x250x100,1e-6,10] \
-f 8x4x2x1 \
-s 3x2x1x0vox \
-o [${3}movingAffine, ${3}movingAffine_deformed.nii.gz] 

**There was no clear improvement over the rigid stage.** Some areas improved others got worse.

What parameters can we change to try and improve this?

### Test padding the target image with 20 voxels:

In [ ]:
# rigid and affine stages combined for testing padding
%%bash -s "$fixed" "$moving" "$out_dir"
file=${1##*/}
ImageMath 3 ${3}${file%.nii.gz}-pad.nii.gz PadImage $1 20

antsRegistration -d 3 \
-u 1 \
-w [0.005,0.995] \
-r [ ${3}${file%.nii.gz}-pad.nii.gz, $2, 1] \
-t Rigid[0.1] \
-m MI[ ${3}${file%.nii.gz}-pad.nii.gz, $2, 1, 32, Regular,0.2] \
-c [1000x500x250x100,1e-6,10] \
-f 8x4x2x1 \
-s 3x2x1x0vox \
-t Affine[0.1] \
-m MI[ ${3}${file%.nii.gz}-pad.nii.gz, $2, 1, 32, Regular,0.2] \
-c [1000x500x250x100,1e-6,10] \
-f 8x4x2x1 \
-s 3x2x1x0vox \
-o [${3}movingAffinePad, ${3}movingAffinePad_deformed.nii.gz] 

ImageMath 3 ${3}movingAffinePad_deformed-depad.nii.gz PadImage ${3}movingAffinePad_deformed.nii.gz -20

The result of this affine was clearly better than the affine without padding.  
It is possible that the rigid was already better and in reality this affine is worse than a padded rigid?  
Does increasing the padding to more than 20 voxels help or does decreasing the padding to, for example 10 still work?

#### Rigid stage of testing padding

In [ ]:
%%bash -s "$fixed" "$moving" "$out_dir"
file=${1##*/}
ImageMath 3 ${3}${file%.nii.gz}-pad.nii.gz PadImage $1 20

antsRegistration -d 3 \
-u 1 \
-w [0.005,0.995] \
-r [ ${3}${file%.nii.gz}-pad.nii.gz, $2, 1] \
-t Rigid[0.1] \
-m MI[ ${3}${file%.nii.gz}-pad.nii.gz, $2, 1, 32, Regular,0.2] \
-c [1000x500x250x100,1e-6,10] \
-f 8x4x2x1 \
-s 3x2x1x0vox \
-o [${3}movingRigidPad, ${3}movingRigidPad_deformed.nii.gz]
       
ImageMath 3 ${3}movingRigidPad_deformed-depad.nii.gz PadImage ${3}movingRigidPad_deformed.nii.gz -20

The result here was very slightly better than without padding.

#### Affine stage of testing padding

In [ ]:
%%bash -s "$fixed" "$moving" "$out_dir"
file=${1##*/}
ImageMath 3 ${3}${file%.nii.gz}-pad.nii.gz PadImage $1 20

antsRegistration -d 3 \
-u 1 \
-w [0.005,0.995] \
-r [${3}movingRigidPad0GenericAffine.mat] \
-t Affine[0.1] \
-m MI[ ${3}${file%.nii.gz}-pad.nii.gz, $2, 1, 32, Regular,0.2] \
-c [1000x500x250x100,1e-6,10] \
-f 8x4x2x1 \
-s 3x2x1x0vox \
-o [${3}movingRigidAffinePad, ${3}movingRigidAffinePad_deformed.nii.gz] 

ImageMath 3 ${3}movingRigidAffinePad_deformed-depad.nii.gz PadImage ${3}movingRigidAffinePad_deformed.nii.gz -20

The result was worse than the rigid BUT potentially less worse than the Affine stage without padding.

### Test padding the target image with 10 voxels:

In [ ]:
%%bash -s "$fixed" "$moving" "$out_dir"
file=${1##*/}
ImageMath 3 ${3}${file%.nii.gz}-pad.nii.gz PadImage $1 10

# Rigid stage of testing padding 10
antsRegistration -d 3 \
-u 1 \
-w [0.005,0.995] \
-r [ ${3}${file%.nii.gz}-pad.nii.gz, $2, 1] \
-t Rigid[0.1] \
-m MI[ ${3}${file%.nii.gz}-pad.nii.gz, $2, 1, 32, Regular,0.2] \
-c [1000x500x250x100,1e-6,10] \
-f 8x4x2x1 \
-s 3x2x1x0vox \
-o [${3}movingRigidPad10, ${3}movingRigidPad10_deformed.nii.gz]
       
ImageMath 3 ${3}movingRigidPad10_deformed-depad.nii.gz PadImage ${3}movingRigidPad10_deformed.nii.gz -10
       
# Affine stage of testing padding 10
antsRegistration -d 3 \
-u 1 \
-w [0.005,0.995] \
-r [${3}movingRigidPad100GenericAffine.mat] \
-t Affine[0.1] \
-m MI[ ${3}${file%.nii.gz}-pad.nii.gz, $2, 1, 32, Regular,0.2] \
-c [1000x500x250x100,1e-6,10] \
-f 8x4x2x1 \
-s 3x2x1x0vox \
-o [${3}movingRigidAffinePad10, ${3}movingRigidAffinePad10_deformed.nii.gz]
       
ImageMath 3 ${3}movingRigidAffinePad10_deformed-depad.nii.gz PadImage ${3}movingRigidAffinePad10_deformed.nii.gz -10

No noticable difference between 10 and 20 voxel padding rigid registration.  
Affine is still slightly worse than rigid but hard to compare to other padding options, i.e., nothing obvious.
### Test padding the target image with 30 voxels:

In [ ]:
%%bash -s "$fixed" "$moving" "$out_dir"
file=${1##*/}
ImageMath 3 ${3}${file%.nii.gz}-pad.nii.gz PadImage $1 30

# Rigid stage of testing padding 30
antsRegistration -d 3 \
-u 1 \
-w [0.005,0.995] \
-r [ ${3}${file%.nii.gz}-pad.nii.gz, $2, 1] \
-t Rigid[0.1] \
-m MI[ ${3}${file%.nii.gz}-pad.nii.gz, $2, 1, 32, Regular,0.2] \
-c [1000x500x250x100,1e-6,10] \
-f 8x4x2x1 \
-s 3x2x1x0vox \
-o [${3}movingRigidPad30, ${3}movingRigidPad30_deformed.nii.gz]
       
ImageMath 3 ${3}movingRigidPad30_deformed-depad.nii.gz PadImage ${3}movingRigidPad30_deformed.nii.gz -30
       
# Affine stage of testing padding 30
antsRegistration -d 3 \
-u 1 \
-w [0.005,0.995] \
-r [${3}movingRigidPad300GenericAffine.mat] \
-t Affine[0.1] \
-m MI[ ${3}${file%.nii.gz}-pad.nii.gz, $2, 1, 32, Regular,0.2] \
-c [1000x500x250x100,1e-6,10] \
-f 8x4x2x1 \
-s 3x2x1x0vox \
-o [${3}movingRigidAffinePad30, ${3}movingRigidAffinePad30_deformed.nii.gz]
       
ImageMath 3 ${3}movingRigidAffinePad30_deformed-depad.nii.gz PadImage ${3}movingRigidAffinePad30_deformed.nii.gz -30

No real difference for rigid but the Affine 30 looks slightly better than 20 or 10 but might be worth checking better.  
**Conclusion: The Affine stage is worse than rigid - at least for this image and for the parameters tested.**

### Test going straight to SyN stage from rigid:

In [ ]:
%%bash -s "$fixed" "$moving" "$out_dir"
file=${1##*/}
ImageMath 3 ${3}${file%.nii.gz}-pad.nii.gz PadImage $1 30
       
# SyN stage padding 30
antsRegistration -d 3 \
-u 1 \
-w [0.005,0.995] \
-r [${3}movingRigidPad300GenericAffine.mat] \
-t SyN[0.1,3,0] \
-m CC[ ${3}${file%.nii.gz}-pad.nii.gz, $2, 1, 4] \
-c [100x70x50x20,1e-6,10] \
-f 8x4x2x1 \
-s 3x2x1x0vox \
-o [${3}movingRigidSyNPad30, ${3}movingRigidSyNPad30_deformed.nii.gz]
       
ImageMath 3 ${3}movingRigidSyNPad30_deformed-depad.nii.gz PadImage ${3}movingRigidSyNPad30_deformed.nii.gz -30

**IT WORKED!!!** Lets put the mask into the original image space:

In [ ]:
%%bash -s "$fixed" "$moving" "$out_dir"
file=${1##*/}
fixedmask='/Volumes/data/prebiostress/data/derivatives/sub-23/ses-4/anat/testManualBrainMask/brainmask.nii.gz'

antsApplyTransforms -d 3 \
       -i $fixedmask \
       -r $2 \
       -o ${3}mask_in_sub01_space.nii.gz \
       -n NearestNeighbor \
       -t [${3}movingRigidSyNPad300GenericAffine.mat,1] \
       -t ${3}movingRigidSyNPad301InverseWarp.nii.gz

This worked good. I am now making a few manual edits to the result.

# Lets test for another brain:

In [ ]:
moving = layout.get(subject="02",modality="anat",session="4",type="T1w",return_type='file')
moving = moving[0]
out_dir = (os.path.dirname(moving)+"/brainmask/")
os.makedirs(out_dir,exist_ok="TRUE")

In [ ]:
%%bash -s "$fixed" "$moving" "$out_dir"
file=${1##*/}
ImageMath 3 ${3}${file%.nii.gz}-pad.nii.gz PadImage $1 30

# Rigid stage of testing padding 30
antsRegistration -d 3 \
-u 1 \
-w [0.005,0.995] \
-r [ ${3}${file%.nii.gz}-pad.nii.gz, $2, 1] \
-t Rigid[0.1] \
-m MI[ ${3}${file%.nii.gz}-pad.nii.gz, $2, 1, 32, Regular,0.2] \
-c [1000x500x250x100,1e-6,10] \
-f 8x4x2x1 \
-s 3x2x1x0vox \
-o [${3}movingRigidPad30, ${3}movingRigidPad30_deformed.nii.gz]
       
ImageMath 3 ${3}movingRigidPad30_deformed-depad.nii.gz PadImage ${3}movingRigidPad30_deformed.nii.gz -30

Pretty good result; lets add the SyN stage.

In [ ]:
%%bash -s "$fixed" "$moving" "$out_dir"
file=${1##*/}
ImageMath 3 ${3}${file%.nii.gz}-pad.nii.gz PadImage $1 30
       
# SyN stage padding 30
antsRegistration -d 3 \
-u 1 \
-w [0.005,0.995] \
-r [${3}movingRigidPad300GenericAffine.mat] \
-t SyN[0.1,3,0] \
-m CC[ ${3}${file%.nii.gz}-pad.nii.gz, $2, 1, 4] \
-c [100x70x50x20,1e-6,10] \
-f 8x4x2x1 \
-s 3x2x1x0vox \
-o [${3}movingRigidSyNPad30, ${3}movingRigidSyNPad30_deformed.nii.gz]
       
ImageMath 3 ${3}movingRigidSyNPad30_deformed-depad.nii.gz PadImage ${3}movingRigidSyNPad30_deformed.nii.gz -30

This worked good. I am now making a few manual edits to the result.

# Lets test for another brain:

In [ ]:
moving = layout.get(subject="03",modality="anat",session="4",type="T1w",return_type='file')
moving = moving[0]
out_dir = (os.path.dirname(moving)+"/brainmask/")
os.makedirs(out_dir,exist_ok="TRUE")
print(fixed)
print(moving)
print(out_dir)

In [ ]:
%%bash -s "$fixed" "$moving" "$out_dir"
file=${1##*/}
ImageMath 3 ${3}${file%.nii.gz}-pad.nii.gz PadImage $1 30

# Rigid stage of testing padding 30
antsRegistration -d 3 \
-u 1 \
-w [0.005,0.995] \
-r [ ${3}${file%.nii.gz}-pad.nii.gz, $2, 1] \
-t Rigid[0.1] \
-m MI[ ${3}${file%.nii.gz}-pad.nii.gz, $2, 1, 32, Regular,0.2] \
-c [1000x500x250x100,1e-6,10] \
-f 8x4x2x1 \
-s 3x2x1x0vox \
-o [${3}movingRigidPad30, ${3}movingRigidPad30_deformed.nii.gz]
       
ImageMath 3 ${3}movingRigidPad30_deformed-depad.nii.gz PadImage ${3}movingRigidPad30_deformed.nii.gz -30

In [ ]:
%%bash -s "$fixed" "$moving" "$out_dir"
file=${1##*/}
ImageMath 3 ${3}${file%.nii.gz}-pad.nii.gz PadImage $1 30
       
# SyN stage padding 30
antsRegistration -d 3 \
-u 1 \
-w [0.005,0.995] \
-r [${3}movingRigidPad300GenericAffine.mat] \
-t SyN[0.1,3,0] \
-m CC[ ${3}${file%.nii.gz}-pad.nii.gz, $2, 1, 4] \
-c [100x70x50x20,1e-6,10] \
-f 8x4x2x1 \
-s 3x2x1x0vox \
-o [${3}movingRigidSyNPad30, ${3}movingRigidSyNPad30_deformed.nii.gz]
       
ImageMath 3 ${3}movingRigidSyNPad30_deformed-depad.nii.gz PadImage ${3}movingRigidSyNPad30_deformed.nii.gz -30

# Skull strip using Ovin2A testing

Try to skull strip using the cropped or un-cropped versions from ovin-2A.


In [ ]:
# create the -g -l inputs for antsJointLabelFusion from a folder

ovin = "/Volumes/data/sheep-brain-atlases/"
t1s = glob.glob(ovin+"*4month_d_crop_08_pad12.nii.gz")
masks = glob.glob(ovin+"*4month_d_crop_08_brainmask_pad12.nii.gz")
atlases = ""
for t1, mask in zip(t1s, masks):
    atlases += ("-g " +t1 +" -l " +mask +" ")


target = "/Volumes/data/prebiostress/data/derivatives/sub-23/ses-4/anat/sub-23_ses-4_T1w_dn_crop2_08_pad12.nii.gz"
output = "/Volumes/data/prebiostress/data/derivatives/sub-23/ses-4/anat/ovin2Atest/"
posterior=output+"Posteriors%04.nii.gz"

cmd = "antsJointLabelFusion.sh -d 3 -t " +target +" -o " +output +" -p " +posterior +" " +atlases +"-y sr -c 2 -j 8 -q 0 > " +output+"log.txt" 
os.system(cmd)

In [ ]:
# create the -g -l inputs for antsJointLabelFusion from a folder

ovin = "/Volumes/data/sheep-brain-atlases/"
t1s = glob.glob(ovin+"*4month_d_crop_08_pad12.nii.gz")
masks = glob.glob(ovin+"*4month_d_crop_08_brainmask_pad12.nii.gz")
atlases = ""
for t1, mask in zip(t1s, masks):
    atlases += ("-g " +t1 +" -l " +mask +" ")


target = "/Volumes/data/prebiostress/data/derivatives/sub-01/ses-4/anat/sub-01_ses-4_T1w_dn_crop-08-pad12.nii.gz"
output = "/Volumes/data/prebiostress/data/derivatives/sub-01/ses-4/anat/ovin2Atest/"
posterior=output+"Posteriors%04.nii.gz"

os.makedirs(output,exist_ok=True)
cmd = "antsJointLabelFusion.sh -d 3 -t " +target +" -o " +output +" -p " +posterior +" " +atlases +"-y sr -c 2 -j 8 -q 0 > " +output+"log.txt" 
os.system(cmd)

In [ ]:
layout()

# Calculate Intracranial Volume

In [2]:
import os
import json
import fnmatch
import glob
import bids
import shutil
import multiprocessing
import pandas as pd
from bids import BIDSLayout
from joblib import Parallel, delayed

derivatives = "/Volumes/data/prebiostress/data/derivatives/"
layout = BIDSLayout(derivatives,validate=False)

grouping = pd.read_csv('participants.tsv', delimiter = '\t')

In [ ]:
for subj in ["01","02"]: #layout.get_subjects():
    i=grouping[grouping["participant_id"].str.contains(subj)].group
    print(subj)
    for ses in layout.get_sessions():
        print(ses)
        targ = os.path.join(derivatives,"sub-"+subj,"ses-"+ses,"anat")
        
        t1path = glob.glob(targ+"/*T1w_dn.nii.gz") # find all T1s   
        t1path = "".join(t1path)
                
        maskpath = glob.glob(targ+"/brainmask-allTemps/Labels.nii.gz") # find all T1s   
        maskpath = "".join(maskpath)
        
        output = targ +"/sub-" +subj +"_ses-" +ses +"_T1w_dn_brainmask.nii.gz"
        
        # reslice the brainmask to the original image header
        cmd= "c3d " +t1path +" " +maskpath +" -reslice-identity -o " +output
        os.system(cmd)
        
        # extract the brain
        cmd= "c3d " +t1path +" " +output +" -multiply -o " +output[:-16] +"brain.nii.gz"
        os.system(cmd)
        
        os.makedirs(targ+"/fsl",exist_ok=True)
        
        # segment the brain
        cmd= "fast -I 4 -B -v -g -o " +targ+"/fsl/" +"sub-" +subj +"_ses-"+ses +" " +t1path
        os.system(cmd)

In [3]:
# fast on the T1
def runeachsess(subj,ses):
    targ = os.path.join(derivatives,"sub-"+subj,"ses-"+ses,"anat")
    t1path = glob.glob(targ+"/*T1w_dn.nii.gz") # find all T1s   
    t1path = "".join(t1path)
    if len(t1path) > 0:
        maskpath = glob.glob(targ+"/brainmask-allTemps/Labels.nii.gz") # find all T1s   
        maskpath = "".join(maskpath)
        
        output = targ +"/sub-" +subj +"_ses-" +ses +"_T1w_dn_brainmask.nii.gz"
        
        # reslice the brainmask to the original image header
        cmd= "c3d " +t1path +" " +maskpath +" -interpolation NearestNeighbor -reslice-identity -o " +output
        #os.system(cmd)
                
        # extract the T1 brain
        cmd= "c3d " +t1path +" " +output +" -multiply -o " +output[:-16] +"brain.nii.gz"
        #os.system(cmd)
        
        t2path = glob.glob(targ+"/*T2w-in-T1w.nii.gz") # find all the T2 already in T1 space 
        t2path = "".join(t2path)
        # extract the T2 brain
        cmd= "c3d " +t2path +" " +output +" -multiply -o " +t2path[:-7] +"_brain.nii.gz"
        #os.system(cmd)
        
        os.makedirs(targ+"/fsl",exist_ok=True)
        # segment the brain
        cmd= "fast -B -b -g -v -o " +targ+"/fsl/" +"sub-" +subj +"_ses-"+ses +" " +output[:-16] +"brain.nii.gz"
        os.system(cmd)
        
for subj in layout.get_subjects():
    print(subj)
    Parallel(n_jobs=4)(delayed(runeachsess)(subj,ses)for ses in layout.get_sessions()) # loop on number of subjects


01
02
03
04
05
06
07
08
09
10
11
12
13
14
15


KeyboardInterrupt: 

In [ ]:
def runeachsess(subj,ses):
    targ = os.path.join(derivatives,"sub-"+subj,"ses-"+ses,"anat")
    t1path = glob.glob(targ+"/*T1w_dn.nii.gz") # find all T1s   
    t1path = "".join(t1path)
    if len(t1path) > 0:        
        output = targ +"/sub-" +subj +"_ses-" +ses +"_T1w_dn_brainmask.nii.gz"
        
        outdir=targ+"/fsl"
        os.makedirs(outdir,exist_ok=True)
        # segment the brain
        #cmd= "fast -I 6 -l 8 -f 0.01 -O 5 -R 0.15 -B -v -g -o " +targ+"/fsl-noBias/" +"sub-" +subj +"_ses-"+ses +" " +output[:-16] +"brain.nii.gz"
        cmd= "fast -B -b -g -v -o " +outdir +"/sub-" +subj +"_ses-"+ses +" " +output[:-16] +"brain.nii.gz"
        os.system(cmd)
        
for subj in ["01","23"]: #layout.get_subjects():
    print(subj)
    Parallel(n_jobs=4)(delayed(runeachsess)(subj,ses)for ses in layout.get_sessions())


# Extract volume data

In [ ]:
# do something to remove existing txt file
!rm seg_vol.csv
!echo "Subject,Group,Session,Total,GM,WM,CSF" > seg_vol.csv

for subj in ["01","02"]: #layout.get_subjects():
    i=grouping[grouping["participant_id"].str.contains(subj)].group
    print(subj)
    for ses in layout.get_sessions():
               
        targ = os.path.join(derivatives,"sub-"+subj,"ses-"+ses,"anat","fsl")
        csf = targ +"/sub-" +subj +"_ses-" +ses +"_pve_0.nii.gz"
        gm = targ +"/sub-" +subj +"_ses-" +ses +"_pve_1.nii.gz"
        wm = targ +"/sub-" +subj +"_ses-" +ses +"_pve_2.nii.gz"
        
        cmd = "fslstats " +csf +" -M -V | awk '{ print $1 * $3  }'"
        csfvol = os.popen(cmd).read()
        
        cmd = "fslstats " +gm +" -M -V | awk '{ print $1 * $3 }'"
        gmvol = os.popen(cmd).read()
        
        cmd = "fslstats " +wm +" -M -V | awk '{ print $1 * $3 }'"
        wmvol = os.popen(cmd).read()
        
        maskpath = glob.glob(targ[:-3]+"*dn_brainmask.nii.gz") # find all T1s   
        maskpath = "".join(maskpath)
        
        cmd = "fslstats " +maskpath +" -V | awk '{ print $2 }'"
        vol = os.popen(cmd).read()
        
        print(csfvol[:-1], gmvol[:-1], wmvol[:-1],vol[:-1])
        

# N4 bias field correction

In [ ]:
def runeachsess(subj,ses):
    targ = os.path.join(derivatives,"sub-"+subj,"ses-"+ses,"anat")
    t1path = glob.glob(targ+"/*T1w_dn_brain.nii.gz") # find all T1s   
    t1path = "".join(t1path)
    if len(t1path) > 0:
        maskpath = glob.glob(targ+"/*dn_brainmask.nii.gz") # find all T1s   
        maskpath = "".join(maskpath)
        
        cmd="N4BiasFieldCorrection -v 1 -d 3 -s 3 -c [100x100x100x100,0.000000001] -b [200] -i " +t1path +" -x " +maskpath +" -o " +t1path[:-7] +"_n4-3.nii.gz"
        os.system(cmd)
        
for subj in ["01","23"]: #layout.get_subjects():
    print(subj)
    Parallel(n_jobs=2)(delayed(runeachsess)(subj,ses)for ses in layout.get_sessions()) # loop on number of subjects